In [1]:
import json
import pandas as pd
import os
import json
import numpy as np
import pickle   
from openai import OpenAI
import os
import multiprocess as mp
from tqdm import tqdm

In [2]:
with open('../data/emb/zhihu_emb.pkl', 'rb') as f:
    raw = pickle.load(f)
    zh_sentences = raw[0]
    zh_emb = raw[1]

In [ ]:
from FlagEmbedding import BGEM3FlagModel
# emb_model = BGEM3FlagModel('BAAI/bge-m3',  
#                        use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
reddit = pd.read_csv('../data/reddit_post.csv')
reddit = reddit[["message_id", "title", "message"]]
reddit.columns = ["message_id","question", "detail"]
reddit_questions = reddit['question'].tolist()
reddit_details = reddit['detail'].tolist()
reddit_sentences = [str(x) + " " + str(y) for x, y in zip(reddit_questions, reddit_details)]


# reddit_emb = emb_model.encode(reddit_sentences)['dense_vecs']

# with open('../data/emb/reddit_emb.pkl', 'wb') as f:
#     pickle.dump([reddit_sentences, reddit_emb], f)

with open('../data/emb/reddit_emb.pkl', 'rb') as f:
    raw = pickle.load(f)
    reddit_sentences = raw[0]
    reddit_emb = raw[1]

similarity = reddit_emb @ zh_emb.T

with open('../data/emb/similarity.pkl', 'wb') as f:
    pickle.dump(similarity, f)


In [2]:
def ir_top5(question, candidates):
        
    client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
    user_prompt = f"# Question:\n{question}\n\n # Candidates:\nA. {candidates[0]}\nB. {candidates[1]}\nC. {candidates[2]}\nD. {candidates[3]}\nE. {candidates[4]}\n\n"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "Given an English question, pick the most similar one from the list of 5 Chinese questions."
                }
            ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_prompt
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
            "name": "similar_question_response",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                "response": {
                    "type": "string",
                    "description": "The letter corresponding to the most similar question.",
                    "enum": [
                    "A",
                    "B",
                    "C",
                    "D",
                    "E"
                    ]
                }
                },
                "required": [
                "response"
                ],
                "additionalProperties": False
            }
            }
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    
    return json.loads(response.choices[0].message.content)["response"]

In [ ]:
reddit = pd.read_csv('../data/reddit_post.csv')
reddit = reddit[["message_id", "title", "message"]]
reddit.columns = ["message_id","question", "detail"]

with open('../data/emb/similarity.pkl', 'rb') as f:
    similarity = pickle.load(f)
    similarity = similarity.T

with open('../data/emb/zhihu_emb.pkl', 'rb') as f:
    raw = pickle.load(f)
    zh_sentences = raw[0]
    zh_emb = raw[1]

def process_row(question_detail, choices):
    """Function to process each row"""
    gpt_pick = ir_top5(question_detail, choices)
    gpt_pick_question = choices['ABCDE'.index(gpt_pick)]
    return gpt_pick, gpt_pick_question



# Example usage
results = []
top_k = 5
list_top_k = []
for i, row in tqdm(enumerate(reddit.itertuples(index=False)), total = reddit.shape[0]): #tqdm(enumerate(reddit.iterrows())):
    top_k_idx = np.argsort(similarity[i])[::-1][:top_k]
    top_k_sim = similarity[i][top_k_idx]
    list_top_k.append([(sim,zh_sentences[j]) for sim, j in zip(top_k_sim,top_k_idx)])
    
    question_detail = str(row.question) + "\n" + str(row.detail)
    choices = [zh_sentences[j] for j in top_k_idx]
    gpt_pick, gpt_pick_question = process_row(question_detail, choices)
    results.append((gpt_pick, gpt_pick_question))

    # without replacement
    zh_sentences = np.delete(zh_sentences, top_k_idx)
    zh_emb = np.delete(zh_emb, top_k_idx, axis=0)
    similarity = np.delete(similarity, top_k_idx, axis=1)

reddit['top_1'] = [x[0][1] for x in list_top_k]
reddit['top_1_sim'] = [x[0][0] for x in list_top_k]
reddit['top_2'] = [x[1][1] for x in list_top_k]
reddit['top_2_sim'] = [x[1][0] for x in list_top_k]
reddit['top_3'] = [x[2][1] for x in list_top_k]
reddit['top_3_sim'] = [x[2][0] for x in list_top_k]
reddit['top_4'] = [x[3][1] for x in list_top_k]
reddit['top_4_sim'] = [x[3][0] for x in list_top_k]
reddit['top_5'] = [x[4][1] for x in list_top_k]
reddit['top_5_sim'] = [x[4][0] for x in list_top_k]

reddit["gpt_pick"], reddit["gpt_pick_question"] = zip(*results)



reddit.to_csv('../data/matched_gpt_4o_mini.csv', index=False)

  0%|          | 0/28809 [00:00<?, ?it/s]

In [ ]:
def sim_check(eng_q, chi_q):
    api_key = os.environ['OPENAI_API_KEY']
    client = OpenAI(api_key = api_key)
    user_prompt = f"# English Question:\n{eng_q}\n\n # Chinese Question:\n{chi_q}\n\n"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "Given an English question and a Chinese question, determine whether they are asking the same question."
                }
            ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_prompt
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
            "name": "similar_question_response",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                "response": {
                    "type": "boolean",
                    "description": "Whether the English question and Chinese question are asking the same question.",
                }
                },
                "required": [
                "response"
                ],
                "additionalProperties": False
            }
            }
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    
    return json.loads(response.choices[0].message.content)["response"]

In [5]:
# reddit["gpt_sim"] = reddit.apply(lambda x: sim_check(str(x['question'])+"\n"+str(x['detail']), x['gpt_pick_question']), axis=1)

reddit = pd.read_csv('../data/matched_gpt_4o_mini.csv')

def sim_check_wrapper(args):
    """ Wrapper function for parallel processing """
    row, sim_check_func = args
    return sim_check_func(str(row['question']) + "\n" + str(row['detail']), row['gpt_pick_question'])

def parallel_apply(df, func, num_workers=4):
    """ Parallel apply using multiprocessing """
    with mp.Pool(num_workers) as pool:
        results = list(tqdm(pool.imap(sim_check_wrapper, [(row, func) for _, row in df.iterrows()]), total=len(df)))
    return results

# Assuming reddit is your DataFrame and sim_check is your function
reddit["gpt_sim"] = parallel_apply(reddit, sim_check, num_workers=mp.cpu_count())
reddit.to_csv('../data/matched_gpt_4o_mini.csv', index=False)

100%|██████████| 28809/28809 [08:21<00:00, 57.42it/s] 


In [11]:
# reddit = pd.read_csv('../data/r_matched_gpt_4o_mini.csv')

count_false = reddit[reddit['gpt_sim'] == False].shape[0]
count_true = reddit[reddit['gpt_sim'] == True].shape[0]
true_ratio = count_true / (count_false + count_true)
print(f"True ratio: {true_ratio}")

True ratio: 0.014623193328050114


In [12]:
reddit[reddit['gpt_sim'] == True].shape

(2325, 13)

In [14]:
reddit[reddit['gpt_sim'] == True].drop_duplicates(subset=['zh_question']).shape

(2325, 13)

In [15]:
reddit[reddit['gpt_sim'] == True].to_csv('../data/r_filtered_gpt_4o_mini.csv', index=False)

In [47]:
# from datasets import load_dataset

# ds = load_dataset("wangrui6/Zhihu-KOL")

data_dict = {}
data_dict['question'] = ds['train']['INSTRUCTION']
data_dict['answer'] = ds['train']['RESPONSE']
for key in json.loads(ds['train']['METADATA'][0]).keys():
    data_dict[key] = [json.loads(x)[key] for x in ds['train']['METADATA']]
df_train = pd.DataFrame(data_dict)

def clean_upvotes(upvote_str):
    upvote_str = upvote_str.replace('赞同', '').strip()
    if len(upvote_str) > 0:
        if "万" in upvote_str:
            upvote_str = upvote_str.replace('万', '').strip()
            return int(float(upvote_str) * 10000)
        else:
            return int(upvote_str)
    else:
        return 0


df_train['upvotes'] = df_train['upvotes'].apply(clean_upvotes)
df_train['answer_id'] = df_train['answer_id'].astype('int64')
df_train['question_id'] = df_train['question_id'].astype('int64')

# df_train = df_train[df_train['url'].notna()] # use only url is not NaN
print(df_train.shape[0])
df_train.head()

1006218


,question,answer,question_id,answer_id,url,upvotes,answer_creation_time
0,怎么说服男朋友买烤箱？,emmmmm，首先想说的是，我买厨房用品一般是不用「说服」的，只是在厨房堆的满满当当的情况下...,357137111,914332816,https://www.zhihu.com/question/357137111/answe...,15,2019-11-28T12:01:22.000Z
1,航天从业者是如何看待电视剧《你是我的荣耀》的？,难得有个关于航天的剧，职场情节悬不悬浮，航天设定和细节走不走心？带着放大镜看了前18集，...,475169837,2053313113,https://www.zhihu.com/question/475169837/answe...,4432,2021-08-11T07:26:08.000Z
2,如何看待PayPal正式进入中国？,PayPal不仅是美国支付巨头，也是国际支付巨头，目前已开拓全球200多个市场，美国以外的市...,348551037,866103409,https://www.zhihu.com/question/348551037/answe...,127,2019-10-22T09:11:15.000Z
3,中金公司交易员月薪八万五是如何做到的？,1、首先，考虑到这位交易员的工作经验，月薪八万五的表述是不正确的：其实是一年的全部薪酬除以1...,545938899,2602363788,https://www.zhihu.com/question/545938899/answe...,450,2022-07-31T13:29:04.000Z
4,摇滚乐（金属）给你们带来了什么？,"ㄟ( ▔, ▔ )ㄏ哪里带来了什么东西啊，除了找到热爱的东西，也失去了很多。听重型现场像疯子...",361437216,1073541478,https://www.zhihu.com/question/361437216/answe...,5,2020-03-12T05:49:28.000Z


In [48]:
# check NaN
df_train[df_train['question_id'].isin([str(x) for x in drop_ids])]

,question,answer,question_id,answer_id,url,upvotes,answer_creation_time


In [52]:
filtered = reddit[reddit['gpt_sim'] == True]
print(filtered.shape[0])
filtered.head()

# merge with train data, left on gpt_pick_question and right on question
# leave only the rows that has most upvotes

merged = pd.merge(filtered, df_train, left_on='zh_question', right_on='question_id', how='left')
merged = merged[merged['upvotes'] == merged.groupby('zh_question')['upvotes'].transform('max')]
merged = merged.drop_duplicates(subset=['zh_question'])
merged = merged[['zh_question', 'answer_id', 'answer', 'upvotes', 'url']]
merged.columns = ['zh_question_id', 'zh_answer_id', 'zh_answer', 'zh_upvotes', 'zh_url']
merged['zh_upvotes'] = merged['zh_upvotes'].astype('int64')
merged['zh_answer_id'] = merged['zh_answer_id'].astype('int64')
# merged = merged[merged['zhihu_upvotes']>0]
print(merged.shape[0])
merged.head()

2325
2325


,zh_question_id,zh_answer_id,zh_answer,zh_upvotes,zh_url
4,352160616,911748450,2022.9月20日更新 其实我在2022年一月已经彻底放弃虾皮哈哈哈哈，老子不玩啦。———...,231,https://www.zhihu.com/question/352160616/answe...
5,570091915,2784599789,1，科学饮食，可多摄入富含优质蛋白的食物，例如多吃肉类、鸡蛋以及豆制品等，还需要多吃新鲜的蔬...,0,https://www.zhihu.com/question/570091915/answe...
11,20948066,701579176,如今伴随着耳机大厂商都各自推出带有主动降噪耳机，一时间降噪耳机似乎变成了一种流行趋势。一副主...,334,https://www.zhihu.com/question/20948066/answer...
27,19849704,232291540,1 12年初到北京，在某小型（却土豪）香港驻京办电影公司工作了三年，期间写了不少东西，却浑浑...,728,https://www.zhihu.com/question/19849704/answer...
34,373924698,1056437024,现在疫情比较严重，更多的时候都是出不了门，还是有一些好玩的，可以在家或在学校做的线上公益。 ...,7,https://www.zhihu.com/question/373924698/answe...


In [53]:
merged.to_csv('../data/zh_q_a.csv', index=False)

In [1]:
# deduplication
import pandas as pd

merged = pd.read_csv('../data/filtered_gpt_4o_mini.csv')
print(merged.shape)
merged = merged.drop_duplicates(subset=['reddit_message_id'])
print(merged.shape)
merged = merged.drop_duplicates(subset=['zhihu_question_id'])
print(merged.shape)

(5397, 8)
(5191, 8)
(1848, 8)
